In [ ]:
from helper import *

config = Config()
config.setFromEnv()

versionControl = 'V1'
promptVersion='P3'
fullName = f'{versionControl}-{promptVersion}'
config.setSaveDetails(fullName)

config.simpleCourseName = 'Movement Science'

In [ ]:
resultsDF = convertPicklesToDF('pickle', config)
errorDF = convertPicklesToDF('error', config)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

filterDF = resultsDF[resultsDF['points_possible']==20]
sns.jointplot(data=filterDF, x='score', y='peerGPT_score_real', kind="reg", marginal_kws=dict(bins=20))
plt.plot([0,20],[0,20], lw=2, color='#414242', linestyle='dashed')
plt.show()


In [ ]:
mergedCriterionData = pd.DataFrame()

for index,row in resultsDF.iterrows():
    criterionData = row['data_peerGPT']
    for col in ['submitter_id', 'assignment_id']:
        criterionData[col] = row[col]
    mergedCriterionData = pd.concat([mergedCriterionData, criterionData])

mergedCriterionData.to_excel('CriterionData.xlsx')

saveDF = resultsDF.copy()
del saveDF['data_peerGPT']
saveDF.to_excel('ScoreData.xlsx')

In [ ]:
mergedCriterionData